In [467]:
from bs4 import BeautifulSoup
import requests
import lxml.html as lh
import pandas as pd

In [469]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json # library to handle JSON files
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
print('Libraries imported.')

Libraries imported.


In [470]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
#Create a handle, page, to handle the contents of the website
page = requests.get(url)
#Store the contents of the website under doc
doc = lh.fromstring(page.content)
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

In [471]:
tr_elements = doc.xpath('//tr')
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print ('%d:%s'%(i,name))
    col.append((name,[]))

1:Postcode
2:Borough
3:Neighbourhood



In [472]:
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 10, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [473]:
[len(C) for (title,C) in col]

[287, 287, 287]

In [475]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)
df.head()

,Postcode,Borough,Neighbourhood\n
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


In [476]:
newdf = df.loc[(df.Borough != "Not assigned")]
newdf.columns=['Postcode','Borough','Neighbourhood']
df1=newdf.groupby("Postcode").agg(lambda x:','.join(set(x)))
df1

,Borough,Neighbourhood
Postcode,,
M1B,Scarborough,"Malvern\n,Rouge\n"
M1C,Scarborough,"Highland Creek\n,Rouge Hill\n,Port Union\n"
M1E,Scarborough,"Morningside\n,Guildwood\n,West Hill\n"
M1G,Scarborough,Woburn\n
M1H,Scarborough,Cedarbrae\n
M1J,Scarborough,Scarborough Village\n
M1K,Scarborough,"Kennedy Park\n,East Birchmount Park\n,Ionview\n"
M1L,Scarborough,"Golden Mile\n,Oakridge\n,Clairlea\n"
M1M,Scarborough,"Cliffcrest\n,Scarborough Village West\n,Cliffs..."


In [477]:
df1.loc[df1['Neighbourhood']=="Not assigned",'Neighbourhood']=df1.loc[df1['Neighbourhood']=="Not assigned",'Borough']
df1['Neighbourhood'] = df1['Neighbourhood'].str.replace('\n','')
df1

,Borough,Neighbourhood
Postcode,,
M1B,Scarborough,"Malvern,Rouge"
M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
M1E,Scarborough,"Morningside,Guildwood,West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae
M1J,Scarborough,Scarborough Village
M1K,Scarborough,"Kennedy Park,East Birchmount Park,Ionview"
M1L,Scarborough,"Golden Mile,Oakridge,Clairlea"
M1M,Scarborough,"Cliffcrest,Scarborough Village West,Cliffside"
